# manipulate database

> Functions to add, delete and update data to the existing structure of the database. Changes to the structure of the Database are handled in the 'create_db' module.

In [ ]:
#| default_exp manipulate_db_contents

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from httpx import get as httpx_get
from fasthtml.common import *

from keybindings_fps.create_db_structure import *

In [ ]:
#|export
def add_binding(db, game_name: str, action_name: str, key_name: str, modifier_name: str = 'tap', description: str = None, sort_order: int = 0):
    """Add a key binding for a specific game and action"""
    game = next(db.t.games.rows_where("name = ?", [game_name]), None)
    if not game:
        raise ValueError(f"Game '{game_name}' not found")
        
    action = next(db.t.actions.rows_where("name = ?", [action_name]), None)
    if not action:
        raise ValueError(f"Action '{action_name}' not found")
        
    key = next(db.t.game_keys.rows_where("name = ?", [key_name]), None)
    if not key:
        raise ValueError(f"Key '{key_name}' not found")
        
    modifier = next(db.t.modifiers.rows_where("name = ?", [modifier_name]), None)
    if not modifier:
        raise ValueError(f"Modifier '{modifier_name}' not found")
        
    return db.t.bindings.insert(dict(
        game_id=game['id'],
        action_id=action['id'],
        key_id=key['id'],
        modifier_id=modifier['id'],
        description=description,
        sort_order=sort_order
    ))

In [ ]:
#| export
def upsert_game(db: database, # Database connection
                name: str, # Name of the game to add to database
                game_type: str = None, # Type of game to add. Currently only 'dumb' and 'tactical' are possible.
                image_url: str = None # URL of the image to add to database, the image will be downloaded and addes as jpg file.
                ):
    """Update existing game or insert new one if it doesn't exist"""
    # Try to find existing game with this name
    existing = db.t.games.rows_where("name = ?", [name])
    game = next(existing, None)  # Get first match or None
    
    if image_url:
        response = httpx_get(image_url)
        if response.is_success:
            image = response.content
    else:
        image = None
    
    game_data = {"name":name, "game_type":game_type, "image":image}

    if game:
        # Update existing game
        game_data['id'] = game['id']
        return db.t.games.update(game_data)
    else:
        # Add new game
        return db.t.games.insert(game_data)


In [ ]:
#| export
def delete_game(db: database, # Database connection, 
                name: str # Name of the game to delete
                ):
    return db.t.games.delete_where("name = ?", [name])

# Example usage for existing database on disk

First connect to the database.

In [ ]:
db = init_db()

In [ ]:
type(db)

apswutils.db.Database

Check for tables and if tables have data.

In [ ]:
print(db.t)
L(db.t.bindings.rows_where("id = ?", [5]))

actions, bindings, categories, game_keys, games, modifiers, sqlite_stat1, sqlite_stat4


(#1) [{'id': 5, 'game_id': 1, 'action_id': 5, 'key_id': 52, 'modifier_id': 1, 'description': None}]

# Add game to database

In [ ]:
db.t.games()[:2]

[{'id': 1, 'name': 'default', 'game_type': 'template', 'image': None},
 {'id': 2,
  'name': 'Insurgency Sandstorm',
  'game_type': 'tactical',
  'image': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02\x1cICC_PROFILE\x00\x01\x01\x00\x00\x02\x0clcms\x02\x10\x00\x00mntrRGB XYZ \x07\xdc\x00\x01\x00\x19\x00\x03\x00)\x009acspAPPL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-lcms\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\ndesc\x00\x00\x00\xfc\x00\x00\x00^cprt\x00\x00\x01\\\x00\x00\x00\x0bwtpt\x00\x00\x01h\x00\x00\x00\x14bkpt\x00\x00\x01|\x00\x00\x00\x14rXYZ\x00\x00\x01\x90\x00\x00\x00\x14gXYZ\x00\x00\x01\xa4\x00\x00\x00\x14bXYZ\x00\x00\x01\xb8\x00\x00\x00\x14rTRC\x00\x00\x01\xcc\x00\x00\x00@gTRC\x00\x00\x01\xcc\x00\x00\x

In [ ]:
upsert_game(db, name='Dummy game', game_type='tactical', image_url='https://assets-prd.ignimgs.com/2021/09/24/insurgency-sandstorm-button-fin-1632454496057.jpg')
db.t.games()[-1]

{'id': 11,
 'name': 'Dummy game',
 'game_type': 'tactical',
 'image': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02\x1cICC_PROFILE\x00\x01\x01\x00\x00\x02\x0clcms\x02\x10\x00\x00mntrRGB XYZ \x07\xdc\x00\x01\x00\x19\x00\x03\x00)\x009acspAPPL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-lcms\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\ndesc\x00\x00\x00\xfc\x00\x00\x00^cprt\x00\x00\x01\\\x00\x00\x00\x0bwtpt\x00\x00\x01h\x00\x00\x00\x14bkpt\x00\x00\x01|\x00\x00\x00\x14rXYZ\x00\x00\x01\x90\x00\x00\x00\x14gXYZ\x00\x00\x01\xa4\x00\x00\x00\x14bXYZ\x00\x00\x01\xb8\x00\x00\x00\x14rTRC\x00\x00\x01\xcc\x00\x00\x00@gTRC\x00\x00\x01\xcc\x00\x00\x00@bTRC\x00\x00\x01\xcc\x00\x00\x00@desc\x00\x00\x00\x00\x00\x00\x00\x03c2\x00\x00\x

## Delete game from database

In [ ]:
delete_game(db, 'Dummy game')
db.t.games()[-1]

{'id': 10,
 'name': 'Overwatch 2',
 'game_type': 'dumb',
 'image': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02\x1cICC_PROFILE\x00\x01\x01\x00\x00\x02\x0clcms\x02\x10\x00\x00mntrRGB XYZ \x07\xdc\x00\x01\x00\x19\x00\x03\x00)\x009acspAPPL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-lcms\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\ndesc\x00\x00\x00\xfc\x00\x00\x00^cprt\x00\x00\x01\\\x00\x00\x00\x0bwtpt\x00\x00\x01h\x00\x00\x00\x14bkpt\x00\x00\x01|\x00\x00\x00\x14rXYZ\x00\x00\x01\x90\x00\x00\x00\x14gXYZ\x00\x00\x01\xa4\x00\x00\x00\x14bXYZ\x00\x00\x01\xb8\x00\x00\x00\x14rTRC\x00\x00\x01\xcc\x00\x00\x00@gTRC\x00\x00\x01\xcc\x00\x00\x00@bTRC\x00\x00\x01\xcc\x00\x00\x00@desc\x00\x00\x00\x00\x00\x00\x00\x03c2\x00\x00\x00\

In [ ]:
delete_game(db, 'Overwatch 2')

<Table games (id, name, game_type, image)>

## Add a new action to the available actions in the database

In [ ]:
#| export
def add_new_action(db: database, # Database connection
                   action: str, # Short description of the action
                   category: str, # Category the action belongs to
                   default_keybinding: str, # Default keybinding for the action
                   default_modifier: str # Default modifier for the action
                   ):
    categories = {c['name']: c['id'] for c in db.t.categories()}

    if category not in categories.keys():
        db.t.categories.insert(dict(name=category, description='wat denk je zelf?'))
        categories = {c['name']: c['id'] for c in db.t.categories()}

    db.t.actions.insert(dict(
            name=action,
            category_id=categories[category]
            ))

    add_binding(db, 'default', action, default_keybinding, default_modifier)

## Add default bindings to game

In [ ]:
#| export
def copy_default_bindings(db, new_game_name: str):
    """Copy all bindings from default game to a new game"""
    # Get the new game
    new_game = next(db.t.games.rows_where("name = ?", [new_game_name]), None)
    if not new_game:
        raise ValueError(f"Game '{new_game_name}' not found")
    
    # Get default game
    default_game = next(db.t.games.rows_where("name = ?", ["default"]), None)
    if not default_game:
        raise ValueError("Default game template not found")
    
    db.t.bindings.delete_where("game_id = ?", [new_game['id']])
    
    # Get all default bindings
    default_bindings = db.t.bindings.rows_where("game_id = ?", [default_game['id']])
    
    # Copy each binding to new game
    for binding in default_bindings:
        db.t.bindings.insert({
            'game_id': new_game['id'],
            'action_id': binding['action_id'],
            'key_id': binding['key_id'],
            'modifier_id': binding['modifier_id'],
            'description': binding['description'],
            'sort_order': binding['sort_order']
        }),

In [ ]:
db = init_db()

In [ ]:
L(db.t.games.rows_where("name = ?", ["default"]))

(#1) [{'id': 1, 'name': 'default', 'game_type': 'template', 'image': None}]

In [ ]:
L(db.t.bindings.rows_where("game_id = ?", [1]))

(#27) [{'id': 1, 'game_id': 1, 'action_id': 1, 'key_id': 23, 'modifier_id': 2, 'description': None},{'id': 2, 'game_id': 1, 'action_id': 2, 'key_id': 19, 'modifier_id': 2, 'description': None},{'id': 3, 'game_id': 1, 'action_id': 3, 'key_id': 1, 'modifier_id': 2, 'description': None},{'id': 4, 'game_id': 1, 'action_id': 4, 'key_id': 4, 'modifier_id': 2, 'description': None},{'id': 5, 'game_id': 1, 'action_id': 5, 'key_id': 52, 'modifier_id': 1, 'description': None},{'id': 6, 'game_id': 1, 'action_id': 6, 'key_id': 3, 'modifier_id': 1, 'description': None},{'id': 7, 'game_id': 1, 'action_id': 7, 'key_id': 51, 'modifier_id': 1, 'description': None},{'id': 8, 'game_id': 1, 'action_id': 8, 'key_id': 55, 'modifier_id': 2, 'description': None},{'id': 9, 'game_id': 1, 'action_id': 9, 'key_id': 49, 'modifier_id': 2, 'description': None},{'id': 10, 'game_id': 1, 'action_id': 10, 'key_id': 65, 'modifier_id': 2, 'description': None},{'id': 11, 'game_id': 1, 'action_id': 11, 'key_id': 66, 'modifie

### Compare bindings of game with the defaults

In [ ]:
#| export
def compare_with_default(db, game_name: str):
    """Compare a game's bindings with default bindings and return differences"""
    # Get both games
    game = next(db.t.games.rows_where("name = ?", [game_name]), None)
    default = next(db.t.games.rows_where("name = ?", ["default"]), None)
    
    if not game:
        raise ValueError(f"Game '{game_name}' not found")
    if not default:
        raise ValueError("Default game template not found")
    
    # Get bindings for both games
    game_bindings = db.t.bindings.rows_where("game_id = ?", [game['id']])
    default_bindings = db.t.bindings.rows_where("game_id = ?", [default['id']])
    
    # Create dictionaries with action_id as key for easy comparison
    game_dict = {b['action_id']: (b['key_id'], b['modifier_id']) for b in game_bindings}
    default_dict = {b['action_id']: (b['key_id'], b['modifier_id']) for b in default_bindings}
    
    # Find differences
    differences = {}
    for action_id, (key_id, mod_id) in game_dict.items():
        if action_id in default_dict and (key_id, mod_id) != default_dict[action_id]:
            differences[action_id] = {
                'game': (key_id, mod_id),
                'default': default_dict[action_id]
            }
    
    return differences

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()